### Install the agentcode sdk

pip install bedrock-agentcore

### Create the skeleton for an agent in dev or prod (cdk & terraform) 

agentcore create

### Run agentcore in development mode in your local environment

agentcore dev

### Test

curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -d '{"prompt": "Hello!"}'

### Deploy to AWS account

agentcore launch

### Interact with the deployed agentcore from local

agentcore invoke '{"prompt":"Tell me a joke"}'

### Modify the agentcore configuration

agentcore configure -e src/main.py

In [ ]:

import boto3
import json
import uuid  
# Initialize the Bedrock AgentCore client
agent_core_client = boto3.client('bedrock-agentcore')
session_id = uuid.uuid4()

agent_arn = ""

prompt = "Tell me a joke"

# Prepare the payload
payload = json.dumps({"prompt": prompt}).encode()
  
# Invoke the agent
response = agent_core_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeSessionId=session_id,
    payload=payload
)
  
  
# Process and print the response
if "text/event-stream" in response.get("contentType", ""):
  
    # Handle streaming response
    content = []
    for line in response["response"].iter_lines(chunk_size=10):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    print("\nComplete response:", "\n".join(content))

elif response.get("contentType") == "application/json":
    # Handle standard JSON response
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    print(json.loads(''.join(content)))
  
else:
    # Print raw response for other content types
    print(response)


### Destroy the deployment

agentcore destroy